<a href="https://colab.research.google.com/github/demolakstate/eGAN_addressing_class_imbalance_with_transfer_learning_on_GAN/blob/main/1to2500_cifar10_dataset_autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 instances of Normal samples - imbalance ratio 1:2500 ##

### Here, we train the discriminator on samples from normal frames ###

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import time, os

In [3]:
print(round(time.time()))

1617527681


In [4]:
time_stamp = round(time.time())

In [5]:
cd /content/gdrive/MyDrive/Anomaly_Detection_in_Images/

/content/gdrive/MyDrive/Anomaly_Detection_in_Images


In [6]:
os.mkdir(str(time_stamp))


In [7]:
os.chdir(str(time_stamp))

In [8]:
time_stamp

1617527681

In [9]:
#!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
tf.__version__

'2.4.1'

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.python.client import device_lib

# "GPU" + device_lib.list_local_devices()[-1].physical_device_desc.split(",")[1]

In [12]:
tf.__version__

'2.4.1'

In [13]:
import glob # The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. 
import imageio # Imageio is a Python library that provides an easy interface to read and write a wide range of image data, including animated images, volumetric data, and scientific formats. 
%matplotlib inline 
import matplotlib.pyplot as plt 
import tensorflow.keras.layers as layers # Keras layers API
import time
from IPython import display # For displaying image

In [14]:
from tensorflow.keras.models import Model

In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
import PIL
import random

In [ ]:
#!pip install -q -U tensorboard
!pip install autokeras

In [17]:
from autokeras import ImageClassifier
from tensorflow import keras

## Load dataset ##

In [18]:
# example of loading the cifar10 dataset
from keras.datasets.cifar10 import load_data
from matplotlib import pyplot
# load the images into memory
(X_train, X_train_label), (X_test, X_test_label) = load_data()
X_train_imbalance = []
#X_train_normal = []

X_train_label_imbalance = []
# y_train_normal = []

count = 1

# plot images from the training dataset
for i in range(len(X_train)):
  if X_train_label[i] == 0 and count <= 2: # few abnormal samples
    X_train_imbalance.append(X_train[i])
    X_train_label_imbalance.append([0])
    count += 1
  elif X_train_label[i] == 1:
    X_train_imbalance.append(X_train[i])
    X_train_label_imbalance.append([1])

In [34]:
# validation data
X_val_balance = []
#X_train_normal = []

X_val_label_balance = []
# y_train_normal = []


# plot images from the training dataset
for i in range(len(X_test)):
  if X_test_label[i] == 0: # few abnormal samples
    X_val_balance.append(X_test[i])
    X_val_label_balance.append([0])
    count += 1
  elif X_train_label[i] == 1:
    X_val_balance.append(X_test[i])
    X_val_label_balance.append([1])

## Normalize the images to the range [0, 1] ##

In [19]:
X_train_imbalance = np.asarray(X_train_imbalance) 
#X_train_abnormal = np.asarray(X_train_abnormal[:2]) # use first 2 samples

In [35]:
X_val_balance = np.asarray(X_val_balance)

In [37]:
X_val_label_balance_np = np.asarray(X_val_label_balance)

In [20]:
len(X_train_imbalance)

5002

In [21]:
#len(X_train_abnormal)

In [22]:
X_train_label_imbalance_np = np.asarray(X_train_label_imbalance)

In [23]:
X_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [24]:
X_train_imbalance[0]

array([[[170, 180, 198],
        [168, 178, 196],
        [177, 185, 203],
        ...,
        [162, 179, 215],
        [158, 178, 214],
        [157, 177, 212]],

       [[168, 181, 198],
        [172, 185, 201],
        [171, 183, 200],
        ...,
        [159, 177, 212],
        [156, 176, 211],
        [154, 174, 209]],

       [[154, 170, 186],
        [149, 165, 181],
        [129, 144, 162],
        ...,
        [161, 178, 214],
        [157, 177, 212],
        [154, 174, 209]],

       ...,

       [[ 74,  84,  80],
        [ 76,  85,  81],
        [ 78,  85,  82],
        ...,
        [ 71,  75,  78],
        [ 68,  72,  75],
        [ 61,  65,  68]],

       [[ 68,  76,  77],
        [ 69,  77,  78],
        [ 72,  79,  78],
        ...,
        [ 76,  80,  83],
        [ 71,  75,  78],
        [ 71,  75,  78]],

       [[ 67,  75,  78],
        [ 68,  76,  79],
        [ 69,  75,  76],
        ...,
        [ 75,  79,  82],
        [ 71,  75,  78],
        [ 73,  77,  80]]

In [25]:
X_train_imbalance = X_train_imbalance.astype('float32') / 255.0

In [38]:
X_val_balance = X_val_balance.astype('float32') / 255.0

## Define the model ##

In [26]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

In [27]:
EPOCHS = 2

In [28]:
classifier = ImageClassifier(seed=9, max_trials=2, metrics=METRICS)

In [29]:
classifier.fit(X_train_imbalance, X_train_label_imbalance_np, epochs=EPOCHS)	

Trial 2 Complete [00h 01m 03s]
val_loss: 8.99029600986978e-07

Best val_loss So Far: 1.1079739209440831e-10
Total elapsed time: 00h 01m 41s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 3s 9ms/step - loss: 0.0304 - tp: 3499.5190 - fp: 2.0000 - tn: 0.0000e+00 - fn: 12.0000 - accuracy: 0.9950 - precision: 0.9993 - recall: 0.9957 - auc: 0.9916
Epoch 2/2
157/157 [==============================] - 1s 9ms/step - loss: 0.0727 - tp: 2541.5190 - fp: 2.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.9978 - precision: 0.9978 - recall: 1.0000 - auc: 0.4662
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [40]:
print(classifier.evaluate(X_val_balance, X_val_label_balance_np))

59/59 [==============================] - 2s 7ms/step - loss: 8.9154 - tp: 876.0000 - fp: 1000.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.4670 - precision: 0.4670 - recall: 1.0000 - auc: 0.5000
[8.915423393249512, 876.0, 1000.0, 0.0, 0.0, 0.4669509530067444, 0.4669509530067444, 1.0, 0.5]


In [41]:
len(X_val_balance)

1876

In [42]:
len(X_val_label_balance_np)

1876

In [31]:
X_test_label

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]], dtype=uint8)

In [32]:
X_train_label_imbalance_np

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [33]:
---

SyntaxError: ignored

In [ ]:
abnormal_data = X_train_abnormal

In [ ]:
train_images = X_train_normal.reshape(X_train_normal.shape[0], 32, 32, 3).astype('float32')
#train_images = (train_images - 127.5) / 127.5

train_images = (train_images) / 255.0


In [ ]:
X_train_normal.shape[0]

In [ ]:
abnormal_data = abnormal_data.astype('float32')
#normal_data = normal_data.astype('float32')

abnormal_data = abnormal_data / 255.0
#normal_data = normal_data / 255.0

In [ ]:
X_test_abnormal = []
X_test_normal = []

# plot images from the training dataset
for i in range(len(X_test)):
  if X_test_label[i] == 0: # few abnormal samples
    X_test_abnormal.append(X_test[i])
  elif X_test_label[i] == 1:
    X_test_normal.append(X_test[i])

In [ ]:
X_validation_abnormal = X_test_abnormal[:500] # use first 500 as abnormal (validation)
X_validation_normal = X_test_normal[:500] # use first 500 as normal (validation)

In [ ]:
# expand to 3d, e.g. add channels dimension
abnormal_data_validation = np.expand_dims(X_validation_abnormal, axis=0)

In [ ]:
abnormal_data_validation = abnormal_data_validation.astype('float32')
abnormal_data_validation = abnormal_data_validation / 255.0

In [ ]:
#abnormal_images = np.array(os.listdir(path_abnormal))
np.random.shuffle(abnormal_data_validation)
BUFFER_SIZE = len(X_validation_abnormal) #200000 # number of images in training i think
BATCH_SIZE = 1#500 # This is just the standard number for batch size. Google for more info
# shuffle and batch the data
np.random.shuffle(abnormal_data_validation)
abnormal_data_validation = np.split(abnormal_data_validation[:BUFFER_SIZE],BATCH_SIZE)

In [ ]:
X_validation_normal = np.asarray(X_validation_normal)

In [ ]:
validation_normal_images = X_validation_normal.reshape(X_validation_normal.shape[0], 32, 32, 3).astype('float32')
#train_images = (train_images - 127.5) / 127.5

validation_normal_images = (validation_normal_images) / 255.0

In [ ]:
validation_normal_ds = tf.data.Dataset.from_tensor_slices(validation_normal_images).shuffle(54077).batch(100000000)

In [ ]:
# X_train_normal_np = X_train_normal_np.astype('float32') / 255.
# X_train_abnormal_np = X_train_abnormal_np.astype('float32') / 255.

In [ ]:
# path = "/content/gdrive/MyDrive/Anomaly_Detection_Videos/normal_frames/"

In [ ]:
# path_abnormal = "/content/gdrive/MyDrive/Anomaly_Detection_Videos/abnormal_frames/"

In [ ]:
# def load_image( infilename ) :
#     img = PIL.Image.open( infilename )
#     #img = img.crop([25,65,153,193])
#     #img = img.resize((64,64))
#     img = img.resize((512,512))
#     data = np.asarray( img, dtype="int32" )
#     return data

In [ ]:
latent_dim = 128
C = 3
img_width = img_height = 32

In [ ]:
IMG_SIZE = (32, 32)

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
base_model_encoder = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

In [ ]:
base_model_encoder.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_encoder.layers))

# Fine-tune from this layer onwards
fine_tune_at_encoder = 422

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_encoder.layers[:fine_tune_at_encoder]:
  layer.trainable =  False

In [ ]:
print(f'{len(base_model_encoder.layers) - fine_tune_at_encoder} layers fine-tuned at generator module')

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
def make_encoder_model():
  inputs = layers.Input(shape=(img_width, img_height, C), name="inputs")
  x = inputs

  # Fine-tuning the base layers

  x = base_model_encoder(inputs, training=False)
  x = global_average_layer(x)
  #x = tf.keras.layers.Dropout(0.2)(x)




  # x = layers.Conv2D(32, (3,3), padding="same")(x)
  # x = layers.BatchNormalization()(x)
  # x = layers.LeakyReLU(alpha=0.2)(x)
  # x = layers.MaxPool2D((2,2))(x)

  # x = layers.Conv2D(32, (3,3), padding="same")(x)
  # x = layers.BatchNormalization()(x)
  # x = layers.LeakyReLU(alpha=0.2)(x)
  # x = layers.MaxPool2D((2,2))(x)

  print('shape after encoder: ', x.shape)

  # x = layers.Flatten()(x)
  units = x.shape[1] * 2
  x = layers.Dense(latent_dim, name="latent")(x)

  return (x, inputs, units)

In [ ]:
def make_generator_model(x, units):
  # Building the generator
  x = layers.Dense(units)(x)
  x = layers.LeakyReLU(alpha=0.2)(x)
  x = layers.Reshape((8, 8, 32))(x) # reshape to shape after encoder

  x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU(alpha=0.2)(x)

  x = layers.Conv2DTranspose(C, (3,3), strides=2, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("sigmoid", name="outputs")(x)

  outputs = x

  return outputs

In [ ]:
x, inputs, units = make_encoder_model()

In [ ]:
outputs = make_generator_model(x=x, units=units)

In [ ]:
encoder_generator_network = Model(inputs=inputs, outputs=outputs)

In [ ]:
print(x.shape)

In [ ]:
abnormal_data.shape

In [ ]:
#abnormal_images = np.array(os.listdir(path_abnormal))
np.random.shuffle(abnormal_data)
BUFFER_SIZE = len(X_train_abnormal) #200000 # number of images in training i think
BATCH_SIZE = 1#500 # This is just the standard number for batch size. Google for more info
# shuffle and batch the data
np.random.shuffle(abnormal_data)
# abnormal_data = np.split(abnormal_data[:BUFFER_SIZE],BATCH_SIZE)

abnormal_data = np.split(abnormal_data[:2500], 1)

In [ ]:
# #abnormal_images = np.array(os.listdir(path_abnormal))
# np.random.shuffle(normal_data)
# BUFFER_SIZE = 5#200000 # number of images in training i think
# BATCH_SIZE = 1#500 # This is just the standard number for batch size. Google for more info
# # shuffle and batch the data
# np.random.shuffle(normal_data)
# #normal_data = np.split(normal_data[:BUFFER_SIZE],BATCH_SIZE)

In [ ]:
train_normal_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(54077).batch(128)

In [ ]:
#len(abnormal_data[0][1])

In [ ]:
#generator = make_generator_model()

#noise = tf.random.normal([1, img_width,img_height, 1]) # shape is 1, 100
generated_image = encoder_generator_network(abnormal_data, training = True)
#plt.imshow(generated_image[0], interpolation="nearest" )
plt.imshow(generated_image[0])

In [ ]:
len(generated_image)

In [ ]:
IMG_SIZE = (32, 32)

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
base_model_disc = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

In [ ]:
base_model_disc.trainable = True

In [ ]:
#base_model.summary()

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_disc.layers))

# Fine-tune from this layer onwards
fine_tune_at_disc = 422

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_disc.layers[:fine_tune_at_disc]:
  layer.trainable =  False

In [ ]:
print(f'{len(base_model_disc.layers) - fine_tune_at_disc} layers fine-tuned at discriminator module')

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
# example of defining the discriminator model
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model


def define_discriminator():
  inputs = tf.keras.Input(shape=(32, 32, 3))
  #x = data_augmentation(inputs)
  #x = preprocess_input(x)
  x = base_model_disc(inputs, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)

  return model


 
# define the standalone discriminator model
def define_discriminator2(in_shape=(img_width, img_height, C)):
 model = Sequential()
 model.add(Conv2D(64, (5,5), strides=(2, 2), padding='same', input_shape=in_shape))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Dropout(0.4))
 model.add(Conv2D(128, (5,5), strides=(2, 2), padding='same'))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Dropout(0.4))
 model.add(Flatten())
 model.add(Dense(1, activation='sigmoid'))
 # compile model
 #opt = Adam(lr=0.0002, beta_1=0.5)
 #model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
 return model
 
# define discriminator model
discriminator = define_discriminator()
# summarize the model
discriminator.summary()
# plot the model
plot_model(discriminator, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
decision = discriminator(generated_image)
print(decision)

In [ ]:
len(decision)

## Testing Abnormal data with discriminator ##

In [ ]:
abn_data = np.expand_dims(abnormal_data[0][-1], axis=0)

In [ ]:
decision = discriminator(abn_data)
print(decision) # discriminator decision should be close to 0

In [ ]:
type(abn_data)

In [ ]:
plt.imshow(tf.squeeze(abn_data))

In [ ]:
train_images[0].shape

## Testing Normal data with discriminator ##

In [ ]:
nor_data = np.expand_dims(train_images[-1], axis=0)

In [ ]:
decision = discriminator(nor_data)
print(decision) # discriminator decision should be close to 1

In [ ]:
plt.imshow(tf.squeeze(nor_data))

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

"""
Discriminator Loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predicitions on real images to an array of 1s
and the dicriminator's predicitons on fake (generated) images to an array of 0s.
"""
@tf.function
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)

  #abnormal_loss = cross_entropy(tf.zeros_like(abnormal_output), abnormal_output)

  #total_loss = real_loss + fake_loss + abnormal_loss
  total_loss = real_loss + fake_loss
  
  return total_loss

"""
Generator Loss

The generator's loss quantifies how well it was able to trick the discrimator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1).
Here, we will compare the discriminators decisions on the generated images to an array of 1s.
"""
@tf.function
def generator_loss(fake_output):
  # pick abnormal image at random for optimization
  i = random.randint(0, len(abnormal_data))
  print(f'i = {i}')

  loss_1 = cross_entropy(tf.ones_like(fake_output), fake_output)
  #loss_2 = cross_entropy(tf.ones_like(abnormal_data[0][i]), abnormal_data[0][i])
  #loss_2 = cross_entropy(tf.ones_like(generated_images[i]), abnormal_data[0][i])
  #loss_2 = cross_entropy(tf.zeros_like(abnormal_output), abnormal_output)
  total_loss = loss_1
  return total_loss

"""
The discriminator and the generator optimizers are different since we will train two networks separately.
The Adam optimization algorithm is an extension to stochastic gradient descent.
Stochastic gradient descent maintains a single learning rate (termed alpha) for all weight updates and the learning rate does not change during training.
A learning rate is maintained for each network weight (parameter) and separately adapted as learning unfolds.

"""
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                 discriminator_optimizer = discriminator_optimizer,
                                 generator = encoder_generator_network,
                                 discriminator = discriminator)

In [ ]:
checkpoint_prefix

In [ ]:
# We will reuse this seed overtime (so it's easier) to visualize progress in the animated GIF
#tf.random.set_seed(1234)
noise_dim = 100
num_examples_to_generate = 16
#seed = tf.random.normal([num_examples_to_generate, noise_dim], seed=1)

In [ ]:
EPOCHS = 4000

In [ ]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
"""
The training loop begins with generator receiving a random seed as input. 
That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). 
The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.
"""

# Notice the use of tf.function
# This annotation causes the function to be "compiled"
def train_step(images, epoch):
    #noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    # take the image links and return a cropped image
    # new_images = []
    # for file_name in images:
    #     #new_pic = load_image( path + file_name)
    #     #new_images.append(new_pic)
    #     new_images.append(file_name)
    
    # images = np.array(new_images)
    #images = images.reshape(images.shape[0], img_width, img_height, C).astype('float32') # puts each number in its own numpy array so instead of [1,2,3] gonna be [[1], [2], [3]]
    #images = (images) / 255 # normalize to [0,1]
    


    """
GradientTape() Records operations for automatic differentiation. Operations are recorded if 
they are executed within this context manager and at least one of their inputs is being "watched".
    """
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = encoder_generator_network(abnormal_data, training=True)

      real_output = discriminator(images, training=True) # images constist of only normal samples
      #abnormal_output = discriminator(abnormal_data, training=True) # abnormal_data constist of only abnormal samples
      fake_output = discriminator(generated_images, training=True)

      #gen_loss = generator_loss(generated_images=generated_images, fake_output=fake_output)
      gen_loss = generator_loss(fake_output=fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, encoder_generator_network.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, encoder_generator_network.trainable_variables)) # The zip() function returns an iterator of tuples based on the iterable object.
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    images = None

    with summary_writer.as_default():
      tf.summary.scalar('gen_loss', gen_loss, step=epoch)
      tf.summary.scalar('disc_loss', disc_loss, step=epoch)

In [ ]:
# ended at 20 epocsh

In [ ]:
  # To plot model performance at various epochs on abnormal and normal samples
  chart_abnormal = []
  chart_normal = []
  chart_epoch = []

In [ ]:
def discriminator_inference(X_validation_abnormal, X_validation_normal, epoch):
  """function to calculate discriminator score on text data"""
  print(f'Abnormal-count{"Normal-count":>17}')
  # print(f'{len(predictions_abnormal_validation[0]):>14}{len(decision_validation_normal[0]):>14}')

  # save model to disk at every epoch
  discriminator.save(f'{time_stamp}/{epoch}.h5')

  predictions_abnormal_validation = []

  # # To plot model performance at various epochs on abnormal and normal samples
  # chart_abnormal = []
  # chart_normal = []
  # chart_epoch = []

  for index, data in enumerate(abnormal_data_validation[0]):
    abn_data_2 = abnormal_data_validation[0][index]
    predictions_abnormal_validation.append(discriminator(abn_data_2))


  count_validation_abnormal = 0

  for ind, _ in enumerate(predictions_abnormal_validation[0]):
    if (tf.keras.backend.get_value(predictions_abnormal_validation[0][ind])) < 0: # setting a threshold of 0.9
      count_validation_abnormal += 1

  print('Abnormal correct count: ', count_validation_abnormal)


  decision_validation_normal = []

  for index, img in enumerate(validation_normal_ds):
    decision = discriminator(img)
    decision_validation_normal.append(decision)
    #print(decision) # discriminator decision should be close to 1


  count_validation_normal = 0

  for ind, _ in enumerate(decision_validation_normal[0]):
    if (tf.keras.backend.get_value(decision_validation_normal[0][ind]))[0] >= 0: # setting a threshold of 0.9
      #print(decision_validation_normal[ind])
      count_validation_normal += 1
  print('Normal correct count: ', count_validation_normal)


  chart_abnormal.append(count_validation_abnormal)
  chart_normal.append(count_validation_normal)
  chart_epoch.append(epoch)

  # return (chart_abnormal, chart_normal, chart_epoch)

In [ ]:
#@tf.function
def train(dataset, epochs):  
  tf.print("Starting man!")
  for epoch in range(epochs):
    start = time.time()
    tf.print("Starting Epoch:", epoch)
    batch_count = 1
    for image_batch in dataset:
      #tf.print("Batch:", batch_count)
      train_step(image_batch, epoch)
      #print("Batch:", batch_count, "Complete")
      batch_count += 1
    

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(encoder_generator_network,
                             epoch + 1,
                             abnormal_data)
    
    tf.print("Epoch:", epoch, "finished")
    tf.print()
    
    # Save the model every epochs
    #checkpoint.save(file_prefix = checkpoint_prefix)
    
    tf.print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    if epoch % EPOCHS == 0:
       checkpoint.save(file_prefix = checkpoint_prefix)  
  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(encoder_generator_network,
                           epochs,
                           abnormal_data)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False. 
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(abnormal_data, training=False).numpy()

  fig = plt.figure(figsize=(150,150))
  print(f'No of predictions: {predictions.shape[0]}')

  #plt.imshow(tf.squeeze(predictions[0]), cmap='gray')

  discriminator_inference(X_test_abnormal, X_test_normal, epoch)
  
  for i in range(1):
      plt.subplot(1, 1, i+1)
      plt.imshow(tf.squeeze(predictions[i]))
      #plt.axis('off')

      # discriminator_inference(X_test_abnormal, X_test_normal, epoch)
      if epoch % 2 == 0:
        pass
        #discriminator_inference(X_test_abnormal, X_test_normal, epoch)
        # plot_model_performance(ch_abnormal, ch_normal, ch_epoch)      
         #plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
from IPython.display import Image
#Image(filename='image_at_epoch_0060.png') 

In [ ]:
type(train_normal_dataset)

In [ ]:
# Launch Tensorboard
%load_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
%%time
train(train_normal_dataset, EPOCHS) # train on first 1000 normal samples

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
!ls ../

## Inferencing Discriminator on X_test ##

In [ ]:
def plot_model_performance(ch_abnormal, ch_normal, ch_epoch):
  """ the function plots abnormal and normal model performance at varying epochs"""
  plt.title('Model performance on test data')
  plt.xlabel('epochs')
  plt.ylabel('samples correctly classified')
  plt.plot(ch_epoch, ch_abnormal, label='Abnormal')
  plt.plot(ch_epoch, ch_normal, label='Normal')
  plt.legend()
  plt.grid()
  plt.show()

In [ ]:
plot_model_performance(chart_abnormal, chart_normal, chart_epoch)

In [ ]:
 print(chart_abnormal)
  

In [ ]:
print(chart_normal)


In [ ]:
print(chart_epoch)

In [ ]:
ch_minority = chart_abnormal
ch_majority = chart_normal
ch_epoch = chart_epoch

In [ ]:
# print(f'EPOCH{"PRECISION":>12}{"RECALL":>7}{"F1-SCORE":>9}')
print(f'EPOCH{"PRECISION":>12}{"RECALL":>7}{"F1-SCORE":>9}')

precision_list = []
recall_list = []
f1_score_list = []


epoch = 0
while epoch < len(ch_minority):
  true_positive = ch_minority[epoch]
  true_negative = ch_majority[epoch]

  false_positive = 1000 - true_negative
  false_negative = 1000 - true_positive

  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)

  f1_score = 2 * ((precision * recall) / (precision + recall))
  precision_list.append(round(precision,2))
  recall_list.append(round(recall, 2))
  f1_score_list.append(round(f1_score, 2))

  # print(f'{epoch+1:>5}{precision:.2f>14}{recall:.2f>9}{f1_score:.2f>11}')
  print(f'{epoch+1:>5}  {precision:.2f}  {recall:.2f}  {f1_score:.2f}')
  epoch += 1

In [ ]:
# precision_list = []
# recall_list = []
# f1_score_list = []

In [ ]:
print(precision_list)

In [ ]:
print(recall_list)

In [ ]:
print(f1_score_list)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.title('Model performance on CIFAR10 test data - imbalance ratio 1:1000')
plt.xlabel('epochs')
plt.ylabel('samples correctly classified')
plt.plot(ch_epoch, precision_list, label='Precision')
plt.plot(ch_epoch, recall_list, label='Recall')
plt.plot(ch_epoch, f1_score_list, label='F1-Score')
plt.legend()
plt.grid()
plt.show()

In [ ]:
max(f1_score_list)

In [ ]:
print(f'maximum F1-score of {max(f1_score_list)} obtained at epoch {1 + (f1_score_list.index(max(f1_score_list)))}')

In [ ]:
max(recall_list)

In [ ]:
print(f'maximum recall of {max(recall_list)} obtained at epoch {1 + (recall_list.index(max(recall_list)))}')

In [ ]:
max(precision_list)

In [ ]:
print(f'maximum precision of {max(precision_list)} obtained at epoch {1 + (precision_list.index(max(precision_list)))}')

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
def plot_model_performance(ch_minority, ch_majority, ch_epoch):
  """ the function plots abnormal and normal model performance at varying epochs"""
  plt.title('Model performance on test data - imbalance ratio 1:1000')
  plt.xlabel('epochs')
  plt.ylabel('samples correctly classified')
  plt.plot(ch_epoch[:8], ch_minority[:8], label='Minority class')
  plt.plot(ch_epoch[:8], ch_majority[:8], label='Majority class')
  plt.legend()
  plt.grid()
  plt.show()

In [ ]:
plot_model_performance(ch_minority, ch_majority, ch_epoch)

In [ ]:
tf.keras.utils.plot_model(discriminator)

In [ ]:
tf.keras.utils.plot_model(encoder_generator_network)

In [ ]:
tf.keras.utils.plot_model(encoder_generator_network, show_shapes=True)

In [ ]:
tf.keras.utils.plot_model(encoder_generator_network, show_shapes=True, rankdir='LR')

In [ ]:
tf.keras.utils.plot_model(encoder_generator_network, rankdir='LR')

In [ ]:
tf.keras.utils.plot_model(discriminator, show_shapes=True, rankdir='LR')

In [ ]:
---

## Evaluate best model with best F1-score on test data ##

## Load best saved model ##

In [ ]:
discriminator = tf.keras.models.load_model('/content/gdrive/MyDrive/Anomaly_Detection_in_Images/1617070888/1617070888/1.h5')

In [ ]:
  # To plot model performance at various epochs on abnormal and normal samples
  chart_abnormal = []
  chart_normal = []
  chart_epoch = []

In [ ]:
def discriminator_inference(X_test_abnormal, X_test_normal):
  """function to calculate discriminator score on text data"""
  print(f'Abnormal-count{"Normal-count":>17}')
  # print(f'{len(predictions_abnormal_test[0]):>14}{len(decision_test_normal[0]):>14}')

  # save model to disk at every epoch
  #discriminator.save(f'{time_stamp}/{epoch}.h5')

  predictions_abnormal_test = []

  # # To plot model performance at various epochs on abnormal and normal samples
  # chart_abnormal = []
  # chart_normal = []
  # chart_epoch = []

  for index, data in enumerate(abnormal_data_test[0]):
    abn_data_2 = abnormal_data_test[0][index]
    predictions_abnormal_test.append(discriminator(abn_data_2))


  count_validation_abnormal = 0

  for ind, _ in enumerate(predictions_abnormal_test[0]):
    if (tf.keras.backend.get_value(predictions_abnormal_test[0][ind])) < 0: # setting a threshold of 0.9
      count_validation_abnormal += 1

  print('Abnormal correct count: ', count_validation_abnormal)


  decision_test_normal = []

  for index, img in enumerate(validation_normal_ds):
    decision = discriminator(img)
    decision_test_normal.append(decision)
    #print(decision) # discriminator decision should be close to 1


  count_validation_normal = 0

  for ind, _ in enumerate(decision_test_normal[0]):
    if (tf.keras.backend.get_value(decision_test_normal[0][ind]))[0] >= 0: # setting a threshold of 0.9
      #print(decision_test_normal[ind])
      count_validation_normal += 1
  print('Normal correct count: ', count_validation_normal)


  chart_abnormal.append(count_validation_abnormal)
  chart_normal.append(count_validation_normal)
  #chart_epoch.append(epoch)

  # return (chart_abnormal, chart_normal, chart_epoch)

In [ ]:
discriminator_inference(X_test_abnormal[500:], X_test_normal[500:]) # evaluate on the remaining test data

In [ ]:
ch_minority = chart_abnormal
ch_majority = chart_normal

In [ ]:
# chart_abnormal[0] = 500


In [ ]:
# chart_normal[0] = 0

In [ ]:
# print(f'EPOCH{"PRECISION":>12}{"RECALL":>7}{"F1-SCORE":>9}')
print(f'EPOCH  {"PRECISION"}{"RECALL":>7} {"F1-SCORE":>9}')

precision_list = []
recall_list = []
f1_score_list = []


epoch = 0
while epoch < len(ch_minority):
    true_positive = ch_minority[epoch]
    true_negative = ch_majority[epoch]

    false_positive = 500 - true_negative
    false_negative = 500 - true_positive

    if true_positive == 0:
        precision = 0
    else:
        precision = true_positive / (true_positive + false_positive)
        
    if true_positive == 0:
        recall = 0
    else:
        recall = true_positive / (true_positive + false_negative)

    if precision == 0 and recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * ((precision * recall) / (precision + recall))
    
    precision_list.append(round(precision,2))
    recall_list.append(round(recall, 2))
    f1_score_list.append(round(f1_score, 2))

    # print(f'{epoch+1:>5}{precision:.2f>14}{recall:.2f>9}{f1_score:.2f>11}')
    print(f'{epoch+1:>5}  {precision:.2f}      {recall:.2f}    {f1_score:.2f}')
    epoch += 1

In [ ]:
---

## Discriminator score on abnormal images ##

In [ ]:
# predictions_abnormal_test = []

# for index, data in enumerate(abnormal_data_test[0]):
#   #abn_data_2 = np.expand_dims(abnormal_data_test[0][index], axis=0)
#   abn_data_2 = abnormal_data_test[0][index]
#   predictions_abnormal_test.append(discriminator(abn_data_2))
#   #print(discriminator(abn_data_2))

In [ ]:
# len(predictions_abnormal_test[0])

In [ ]:
# count_test_abnormal = 0

# for ind, _ in enumerate(predictions_abnormal_test[0]):
#   if (tf.keras.backend.get_value(predictions_abnormal_test[0][ind])) < 0: # setting a threshold of 0.9
#     count_test_abnormal += 1
# print('Abnormal correct count: ', count_test_abnormal)

In [ ]:
# print(f'{count_test_abnormal} abnormal missed' )

In [ ]:
# decision_test_normal = []

# for index, img in enumerate(test_normal_ds):
#   #nor_data_test = np.expand_dims(test_normal_images[int(index)], axis=0)
#   #nor_data_test = test_normal_ds

#   decision = discriminator(img)
#   decision_test_normal.append(decision)
#   #print(decision) # discriminator decision should be close to 1

In [ ]:
# len(decision_test_normal[0])

In [ ]:
(tf.keras.backend.get_value(decision_test_normal[0][0]))[0]

In [ ]:
# count_test_normal = 0

# for ind, _ in enumerate(decision_test_normal[0]):
#   if (tf.keras.backend.get_value(decision_test_normal[0][ind]))[0] >= 0: # setting a threshold of 0.9
#     #print(decision_test_normal[ind])
#     count_test_normal += 1
# print('Normal correct count: ', count_test_normal)

In [ ]:
# print(f'{len(decision_test_normal[0]) - count_test_normal} normal missed' )

## Save models ##

In [ ]:
#encoder_generator_network.save('encoder_generator.h5')

In [ ]:
#discriminator.save('discriminator.h5')

In [ ]:
----

In [ ]:
fig = plt.figure(figsize=(4,4))
predictions = checkpoint.generator(x_abnormal, training = False).numpy()
for i in range(seed.shape[0]):
    plt.subplot(4,4,i+1)
    plt.imshow(predictions[i], interpolation="nearest")
    plt.axis('off')
    
plt.show()

In [ ]:
noise = tf.random.normal([1,100]) # shape is 1, 100
random_face = checkpoint.generator(noise, training = False).numpy()[0]
plt.imshow(random_face, interpolation='nearest')
plt.axis("off")
plt.show()

In [ ]:
%ls

In [ ]:
with imageio.get_writer('dcgan.gif', mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# A hack to display the GIF inside this notebook
os.rename('dcgan.gif', 'dcgan_celebA.gif.png')

In [ ]:
display.Image(filename="dcgan_celebA.gif.png")

In [ ]:
%ls training_checkpoints/

%cp image_at_epoch_0001.png play-with-faces/